In [1]:
from stance_gator.data_modules import StanceCorpus, StanceDataModule
from stance_gator.span_module import SpanModule
from stance_gator.torch_utils import load_module
from stance_gator.constants import TriStance
from lightning.fabric.utilities.apply_func import move_data_to_device
import torch
import numpy as np
from scipy.special import softmax

import os
import glob

In [2]:
import html
from IPython.display import HTML, display

In [21]:
# ckpt_path = '/home/ethanlmines/blue_dir/experiments/lightning_logs/30MayNesyStance/checkpoints/epoch=03-val_macro_f1=0.761.ckpt'
# ckpt_path = '/home/ethanlmines/blue_dir/experiments/lightning_logs/31MayQuarterTemp/checkpoints/epoch=03-val_macro_f1=0.757.ckpt'
# ckpt_path = '/home/ethanlmines/blue_dir/experiments/lightning_logs/31MayQuarterTemp/checkpoints/epoch=03-val_macro_f1=0.757.ckpt'
# checkpoint_dir = "/home/ethanlmines/blue_dir/experiments/lightning_logs/2JunSpanMod_Des25/checkpoints/"
checkpoint_dir = "/home/ethanlmines/blue_dir/experiments/lightning_logs/2JunSpanMod_Des50/checkpoints/"
ckpt_path = glob.glob(os.path.join(checkpoint_dir, "*.ckpt"))[0]
span_mod: SpanModule  = load_module(ckpt_path)

You are using a model of type bert to instantiate a model of type bert_for_stance. This is not supported for all configurations of models and can yield errors.
Some weights of BertForStance were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.1.bias', 'classifier.1.weight', 'classifier.3.bias', 'classifier.3.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
data_mod = StanceDataModule(
    [StanceCorpus(
        path="/home/ethanlmines/blue_dir/datasets/VAST/vast_zero_dev.csv",
        corpus_type='vast',
        data_ratio=(0, 0, 1)
    )]
)
data_mod.encoder = span_mod.encoder
data_mod.setup('predict')

Parsing /home/ethanlmines/blue_dir/datasets/VAST/vast_zero_dev.csv: 0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Parsing /home/ethanlmines/blue_dir/datasets/VAST/vast_zero_dev.csv: 1019it [00:03, 332.25it/s]
/home/ethanlmines/blue_dir/conda_envs/stance_gator_dev/lib/python3.12/site-packages/torch/utils/data/dataset.py:473: UserWarning: Length of split at index 0 is 0. This might result in an empty dataset.
  warnings.warn(
/home/ethanlmines/blue_dir/conda_envs/stance_gator_dev/lib/python3.12/site-packages/torch/utils/data/dataset.py:473: UserWarning: Length of split at index 1 is 0. This might result in an empty dataset.
  warnings.warn(


In [23]:
span_mod.eval().to('cuda')
tokenizer = span_mod.encoder.tokenizer
i = 0

def toks_to_html_str(toks):
    return html.escape(tokenizer.convert_tokens_to_string(toks))

class_prob_prefix = str([f"P({s.name})" for s in TriStance])

special_ids = set(tokenizer.all_special_ids)
for batch in data_mod.test_dataloader():
    batch = move_data_to_device(batch, span_mod.device)
    labels = batch.pop('labels')
    output: SpanModule.Output = span_mod(**batch)


    iterator = zip(
                    labels.detach().cpu().tolist(),
                    batch['input_ids'].detach().tolist(),
                    output.start_inds.detach().cpu().tolist(),
                    output.stop_inds.detach().cpu().tolist(),
                    output.first_pass.logits.detach().cpu().numpy(),
                    output.second_pass.logits.detach().cpu().numpy(),
    )
    for label_id, id_list, start_index, stop_index, first_logits, second_logits in iterator:

        context_ids = []
        target_ids = []
        span_start = None
        span_end = None
        i = 0
        while i < len(id_list) and id_list[i] in special_ids:
            i += 1
        while i < len(id_list) and id_list[i] not in special_ids:
            if i == start_index:
                span_start = i
            if i == stop_index:
                span_end = i
            context_ids.append(int(id_list[i]))
            i += 1
        while i < len(id_list) and id_list[i] in special_ids:
            i += 1
        while i < len(id_list) and id_list[i] not in special_ids:
            target_ids.append(int(id_list[i]))
            i += 1
        assert span_start is not None and span_end is not None

        html_toks = []

        target_str = tokenizer.decode(target_ids)
        context_toks = tokenizer.convert_ids_to_tokens(context_ids)

        pre_span  = toks_to_html_str(context_toks[:span_start])
        span      = toks_to_html_str(context_toks[span_start:span_end + 1])
        post_span = toks_to_html_str(context_toks[span_end + 1:])

        html_toks.append(f'<p> <strong>Target</strong>: {target_str} </p>')
        html_toks.append(f'<p> <strong>Sample</strong>: {pre_span} <mark>{span}</mark> {post_span} </p>')


        format_numlist = lambda x: [f"{round(v, 3):.3f}" for v in x]

        html_toks.append(f'<p> <strong>Ground Truth</strong>: {TriStance(label_id).name} </p>')
        html_toks.append(f'<p> <strong>Prediction from whole doc</strong>: {class_prob_prefix} = {format_numlist(softmax(first_logits))} </p>')
        html_toks.append(f'<p> <strong>Prediction from highlighted span</strong>: {class_prob_prefix} = {format_numlist(softmax(second_logits))} </p>')

        html_str = "".join(html_toks)
        display(HTML(html_str))
        exit = input("Press enter to continue (q to quit)").lower() == 'q'
        if exit:
            break
    if exit:
        break

In [6]:
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'token_type_ids', 'context_mask'])

In [11]:
# html_str = '<table><tbody> <tr> <td colspan="2">Doof</td> </tr> <tr> <td>Hi</td> <td>Yo</td> <td>Wassup</td> </tbody></table>'
display(HTML(html_str))

Token,Att,Sent,Contribution
modest,0.07361588627099991,0.7244386076927185,0.0533301904797554
female,0.045711301267147064,0.7637415528297424,0.0349116213619709
##ital,0.04429059103131294,0.7480758428573608,0.03313272073864937
male,0.04917573928833008,0.5769947171211243,0.028374141082167625
sexual,0.03629869967699051,0.734909176826477,0.026676246896386147
sexuality,0.034179843962192535,0.7561362385749817,0.025844618678092957
women,0.03435264900326729,0.7336478233337402,0.02520274557173252
women,0.02912239544093609,0.6735926866531372,0.019616631790995598
family,0.03144524246454239,0.5897860527038574,0.01854596473276615
male,0.034589290618896484,0.5014801621437073,0.017345843836665154


In [9]:
0.0736 * 0.724

0.0532864